In [1]:

from system import System

system = System("/Users/emanuelwreeby/Plugg/Terminer/Exjobb/src/data/univrses/record_2022-04-22_07-17-55.hdf5")


In [2]:
system.load_detections()

In [3]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'top', 'bottom', 'left', 'right'}
    bb2 : dict
        Keys: {'top', 'bottom', 'left', 'right'}


    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['left'] < bb1['right']
    assert bb1['top'] < bb1['bottom']
    assert bb2['left'] < bb2['right']
    assert bb2['top'] < bb2['bottom']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['left'], bb2['left'])
    y_top = max(bb1['top'], bb2['top'])
    x_right = min(bb1['right'], bb2['right'])
    y_bottom = min(bb1['bottom'], bb2['bottom'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['right'] - bb1['left']) * (bb1['bottom'] - bb1['top'])
    bb2_area = (bb2['right'] - bb2['left']) * (bb2['bottom'] - bb2['top'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [12]:
system.detections[:20]

array([(659517232112, 29, [ 0,  2], 0.640625  , 459, 501,  293,  363, 659517232112),
       (659583724787, 29, [ 0,  3], 0.88671875, 457, 500,  296,  367, 659583724787),
       (659583724787, 29, [ 0,  4], 0.62109375, 454, 502,  157,  277, 659583724787),
       (659616971125, 29, [ 0,  6], 0.85546875, 454, 499,  293,  368, 659616971125),
       (659616971125, 29, [ 0,  7], 0.640625  , 452, 502,  156,  277, 659616971125),
       (659650217707, 29, [ 0,  9], 0.83984375, 455, 499,  293,  366, 659650217707),
       (659650217707, 29, [ 0, 11], 0.671875  , 460, 534, 1089, 1259, 659650217707),
       (659683523696, 29, [ 0, 12], 0.83984375, 454, 499,  300,  369, 659683523696),
       (659683523696, 29, [ 0, 13], 0.80859375, 455, 535, 1087, 1261, 659683523696),
       (659683523696, 29, [ 0, 15], 0.6875    , 473, 537,  982, 1091, 659683523696),
       (659716829684, 29, [ 0, 16], 0.828125  , 454, 498,  300,  370, 659716829684),
       (659716829684, 29, [ 0, 18], 0.65625   , 456, 534, 1080, 1

In [4]:
import matplotlib.pyplot as plt
n_detected_vehicles = 0

detected_vehicles = []
%matplotlib qt

for detection in system.detections[:100]:
    placed = False
    for i in range(len(detected_vehicles)):
        vehicle = detected_vehicles[i]
        # Calculate IoU
        # Show image and draw bounding box
        image = system.get_image_at_timestamp(detection["timestamp"])
        plt.imshow(image)
        plt.gca().add_patch(plt.Rectangle((detection["left"], detection["top"]), detection["right"] - detection["left"], detection["bottom"] - detection["top"], fill=False, color="red"))
        plt.gca().add_patch(plt.Rectangle((vehicle["last_position"]["left"], vehicle["last_position"]["top"]), vehicle["last_position"]["right"] - vehicle["last_position"]["left"], vehicle["last_position"]["bottom"] - vehicle["last_position"]["top"], fill=False, color="blue"))
        plt.show()

        iou = get_iou(detection[["top", "bottom", "left", "right"]], vehicle["last_position"][["top", "bottom", "left", "right"]])
        if iou > 0.7 and detection["timestamp"] - vehicle["latest_detection"] < .5e9: # Half a second
            # Add to vehicle
            

            vehicle["detection_uuids"].append(detection["uuid"][-1])
            placed = True
            break
    if not placed:
        # Create new vehicle
        n_detected_vehicles += 1
        detected_vehicles.append({
            "vehicle_nr": n_detected_vehicles,
            "detection_uuids": [detection["uuid"][-1]],
            "last_position": detection[["top", "bottom", "left", "right"]],
            "latest_detection": detection["timestamp"]
        })


        


: 

In [16]:
detected_vehicles

[{'vehicle_nr': 1,
  'detection_uuids': [2, 3, 6, 9, 12],
  'last_position': (459, 501, 293, 363),
  'latest_detection': 659517232112},
 {'vehicle_nr': 2,
  'detection_uuids': [4, 7, 21, 24, 26, 30, 33, 37],
  'last_position': (454, 502, 157, 277),
  'latest_detection': 659583724787},
 {'vehicle_nr': 3,
  'detection_uuids': [11, 13, 18, 20, 23, 44],
  'last_position': (460, 534, 1089, 1259),
  'latest_detection': 659650217707},
 {'vehicle_nr': 4,
  'detection_uuids': [15],
  'last_position': (473, 537, 982, 1091),
  'latest_detection': 659683523696},
 {'vehicle_nr': 5,
  'detection_uuids': [16, 19, 22, 25, 28, 31, 32, 38],
  'last_position': (454, 498, 300, 370),
  'latest_detection': 659716829684},
 {'vehicle_nr': 6,
  'detection_uuids': [29, 35, 39, 42, 47],
  'last_position': (449, 498, 181, 288),
  'latest_detection': 659850003409},
 {'vehicle_nr': 7,
  'detection_uuids': [34, 36, 43],
  'last_position': (453, 499, 310, 384),
  'latest_detection': 659949829848},
 {'vehicle_nr': 8,
